In [ ]:
#! pip install requests pymongo dnspython beautifulSoup4
import feedparser
import dateutil.parser
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import mongo_utils
from datetime import datetime as dt

In [ ]:
dbname = mongo_utils.get_database()

In [ ]:
urls = ['http://news.bnt.bg/bg/rss/news.xml',
        'http://news.bnt.bg/bg/rss/blogs.xml',
    ]
feeds = [feedparser.parse(url)['entries'] for url in urls]
feeds[1][0].keys()
feed = [item for feed in feeds for item in feed]

In [ ]:
for newslink in feed:
  try:
    #print(newslink["id"])
    nlink=newslink["id"]
    #print(nlink)
    dt_obj=newslink["published"]
    date_posted = dt.strptime(dt_obj, '%a, %d %b %Y %H:%M:%S %z').timestamp()
    news_title=newslink["title"]
    page = requests.get(nlink)
    data = page.text
    soup = BeautifulSoup(data, 'html.parser')
    #text = soup.find_all('p')
    text= soup.find('div', class_='txt-news').get_text()
    text= text.replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')
    news_payload = {
    "date_posted" : date_posted,
    "news_link" : nlink,
    "news_text" : text,
    "news_title" : news_title
    }
    print(news_payload)
    collection_name = dbname["bnt_scrape_1"]
    collection_name.insert_many([news_payload])
  except:
    pass